In [ ]:
import json
from functools import partial

import torch
from datasets import load_dataset
from transformers import (
    AutoProcessor,
    DonutProcessor, VisionEncoderDecoderModel,
    Trainer,
    TrainingArguments,
)
import datetime
from transformers import VisionEncoderDecoderConfig


max_length = 164
image_size = [2560, 1920]

model_path = '/home/ubuntu/donut/checkpoint-8500/'

# update image_size of the encoder
# during pre-training, a larger image size was used
config = VisionEncoderDecoderConfig.from_pretrained(model_path)
config.encoder.image_size = image_size # (height, width)
# update max_length of the decoder (for generation)
config.decoder.max_length = max_length
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base", config=config)
model = VisionEncoderDecoderModel.from_pretrained(model_path, config=config)
model.eval()

# Donut requires that
model.config.decoder_start_token_id = processor.tokenizer.bos_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id


In [ ]:
from datasets import load_dataset

dataset = load_dataset("arnaudstiegler/synthetic_us_passports_easy")

In [ ]:
model.to('cuda')

In [ ]:
inp = processor(dataset['test'][1]['image'].convert('RGB'), return_tensors='pt')
out = model.generate(pixel_values=inp['pixel_values'].to('cuda'))

In [ ]:
print(processor.batch_decode(out))
print(dataset['test'][1])